# L5: Automate Event Planning

In this lesson, you will learn more about Tasks.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Crew, Task

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key,get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool() # surf internet
scrape_tool = ScrapeWebsiteTool() # surf website

## Creating Agents

In [5]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    )
)

In [6]:
 # Agent 2: Logistics Manager
logistics_manager = Agent(
    role='Logistics Manager',
    goal=(
        "Manage all logistics for the event "
        "including catering and equipmen"
    ),
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    )
)

In [7]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    )
)

## Creating Venue Pydantic Object

- Create a class `VenueDetails` using [Pydantic BaseModel](https://docs.pydantic.dev/latest/api/base_model/).
- Agents will populate this object with information about different venues by creating different instances of it.

In [8]:
from pydantic import BaseModel
# Define a Pydantic model for venue details 
# (demonstrating Output as Pydantic)
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

## Creating Tasks
- By using `output_json`, you can specify the structure of the output you want.
- By using `output_file`, you can get your output in a file.
- By setting `human_input=True`, the task will ask for human feedback (whether you like the results or not) before finalising it.

In [9]:
venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails, 
    output_file="venue_details.json",  
      # Outputs the venue details as a JSON file
    agent=venue_coordinator
)

- By setting `async_execution=True`, it means the task can run in parallel with the tasks which come after it.

In [10]:
logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True, ###### with human input
    async_execution=True, # process parallel 
    agent=logistics_manager
)

In [11]:
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    async_execution=True,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent
)

## Creating the Crew

**Note**: Since you set `async_execution=True` for `logistics_task` and `marketing_task` tasks, now the order for them does not matter in the `tasks` list.

In [12]:
# Define the crew with agents and tasks
event_management_crew = Crew(
    agents=[venue_coordinator, 
            logistics_manager, 
            marketing_communications_agent],
    
    tasks=[venue_task, 
           logistics_task, 
           marketing_task],
    
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [13]:
event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_description': "A gathering of tech innovators "
                         "and industry leaders "
                         "to explore future technologies.",
    'event_city': "San Francisco",
    'tentative_date': "2024-09-15",
    'expected_participants': 500,
    'budget': 20000,
    'venue_type': "Conference Hall"
}

**Note 1**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

**Note 2**: 
- Since you set `human_input=True` for some tasks, the execution will ask for your input before it finishes running.
- When it asks for feedback, use your mouse pointer to first click in the text box before typing anything.

In [14]:
result = event_management_crew.kickoff(inputs=event_details)

 [DEBUG]: == Working Agent: Venue Coordinator
 [INFO]: == Starting Task: Find a venue in San Francisco that meets criteria for Tech Innovation Conference.


> Entering new CrewAgentExecutor chain...
I need to start by researching potential venues in San Francisco that would be suitable for the Tech Innovation Conference.

Action: Search the internet
Action Input: {"search_query": "Tech Innovation Conference venues in San Francisco"} 


Search results: Title: Top 12 tech conferences in San Francisco for 2024 - TravelPerk
Link: https://www.travelperk.com/blog/top-tech-conferences-in-san-francisco/
Snippet: Top 12 tech conferences in San Francisco for 2024 · Developer Week · Women Impact Tech · SaaS Connect · Cybersecurity and Cloud Congress · Startup ...
---
Title: Top 16 Must-Visit San Francisco Tech Events in 2024
Link: https://startupvoyager.com/tech-events-san-francisco/
Snippet: 1. Tech Summit Silicon Valley ... Tech Summit Silicon Valley is an immersive two-day event offering a ble

This is the agent final answer: After researching potential venues in San Francisco for the Tech Innovation Conference, the top 12 tech conferences in San Francisco for 2024 were identified. These conferences include Developer Week, Women Impact Tech, SaaS Connect, Cybersecurity and Cloud Congress, Startup Grind Global Conference, Dreamforce, San Francisco Tech Summit, TECHSPO Silicon Valley, SaaStr Annual, TechCrunch Disrupt, ProductWorld, and Transform 2024.

Based on the information gathered, it seems that the San Francisco Tech Summit would be a suitable venue for the Tech Innovation Conference. The event is scheduled for June 19, 2024, at the Santa Clara Convention Center in Santa Clara. With over 200 speakers, the summit offers an action-packed agenda for inspiration, insights, innovation, and connections. It also provides networking opportunities with innovators and decision-makers from across the industry, masterclasses, workshops, and clinics to strengthen knowledge, and Artif

Thought: I need to gather more specific information about the Tech Innovation Conference to create a marketing plan.
Action: Search the internet
Action Input: {"search_query": "Tech Innovation Conference 2024 details"} 


Search results: Title: TechConnect World 2024
Link: https://techconnectworld.com/
Snippet: TechConnect World 2024 will take place June 17-19, 2024, Washington DC, Gaylord National Harbor.
---
Title: Tech Conferences: The Best Tech Events Guide for 2024 - Bizzabo
Link: https://www.bizzabo.com/blog/technology-events
Snippet: This carefully curated conference directory features more than 100 tech conferences and events happening globally in 2024. Check it out!
---
Title: 2024 Cybersecurity & Technology Innovation Conference
Link: https://www.energy.gov/cio/2024-cybersecurity-technology-innovation-conference
Snippet: Date/Location: May 13-16, 2019, in Denver, CO. Theme: “Powering Cybersecurity and Information Technology Innovation”. 2018. Date/Location: June 4-7, 2018 in 

Exception in thread Thread-5 (_execute):
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/usr/local/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.11/site-packages/crewai/task.py", line 182, in _execute
    result = agent.execute_task(
             ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/crewai/agent.py", line 221, in execute_task
    result = self.agent_executor.invoke(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/langchain/chains/base.py", line 163, in invoke
    raise e
  File "/usr/local/lib/python3.11/site-packages/langchain/chains/base.py", line 153, in invoke
    self._call(inputs, run_manager=run_manager)
  File "/us

Thought: I need to gather more specific information about the Tech Innovation Conference to create a marketing plan.
Action: Read website content
Action Input: {"website_url": "https://inevent.com/blog/others/top-20-tech-conferences-to-attend-in-2024.html"} 

Top 20 Tech Conferences to Attend in 2024 | InEvent
|
Blog
Agency
Eventland
Hospitality
Events
Interviews
Logistics
Marketing
Press
Tech & Trends
Venues
Others
Solutions
In-Person Events
Virtual & Hybrid Events
Event Management
Live Broadcasting
Hospitality
Live Studio
Webinars
Virtual Lobby
Accessibility
Integrations
Industries
Financial
Pharmaceutical
Government
Agency
Education
Technology
Automotive
Investor Relations
Experience Center
Event Planner
Travel Manager
IT Manager
Marketing Manager
Sales Manager
HR Partner
Tribal Council
Pricing
Customers
Company
About us
Why InEvent?
Jobs
EventAcademy
Diversity & Inclusion
EventLand Community
Sustainability (ESG)
InEvent TV Shows
Events
Book a meeting
Login
Top 20 Tech Conferences t

Thought: I need to gather more specific information about the Tech Innovation Conference to create a marketing plan.
Action: Read website content
Action Input: {"website_url": "https://inevent.com/blog/others/top-20-tech-conferences-to-attend-in-2024.html"} 

I tried reusing the same input, I must stop using this action input. I'll try something else instead.



Thought: I need to gather more specific information about the Tech Innovation Conference to create a marketing plan.
Action: Read website content
Action Input: {"website_url": "https://rdtechinnovation.com/"} 

Real Deals Tech Innovation
Home 
Agenda 
Advisory Board 
Speakers 
Sponsors 
Contact 
Your Account 
REGISTER INTEREST FOR 2025
 Real Deals Tech Innovation conference 21 March 2024The King's Fund, No. 11 Cavendish Square, London Gaining a critical edge through the next wave of digital adoption 
REGISTER INTEREST FOR 2025
Real Deals’ fourth annual Tech Innovation conference, co-hosted with
	The Drawdown, is taking place on

- Display the generated `venue_details.json` file.

In [15]:
import json
from pprint import pprint

with open('venue_details.json') as f:
   data = json.load(f)

pprint(data)

{'address': '5001 Great America Pkwy, Santa Clara, CA 95054',
 'booking_status': 'Available',
 'capacity': 200,
 'name': 'Santa Clara Convention Center'}


- Display the generated `marketing_report.md` file.

**Note**: After `kickoff` execution has successfully ran, wait an extra 45 seconds for the `marketing_report.md` file to be generated. If you try to run the code below before the file has been generated, your output would look like:

```
marketing_report.md
```

If you see this output, wait some more and than try again.

In [16]:
from IPython.display import Markdown
Markdown("marketing_report.md")

marketing_report.md